In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("D:\healthcare chatbot\End-to-end-Medical-Chatbot-using-Llama2\data")

In [7]:
# extracted_data

In [5]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


In [11]:
# text_chunks

In [11]:
pip install sentence_transformers


Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:

from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer



embeddings =download_hugging_face_embeddings()


d:\healthcare chatbot\End-to-end-Medical-Chatbot-using-Llama2\mchatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ankit\AppData\Local\Temp\ipykernel_25952\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
# query_result

In [14]:
pip install pinecone

  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:


pip install google protobuf grpcio grpcio-status


  Using cached grpcio-1.73.1-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
Using cached grpcio-1.73.1-cp311-cp311-win_amd64.whl (4.3 MB)

   ----- ---------------------------------- 1/7 [protobuf]
   ----------- ---------------------------- 2/7 [grpcio]
   ----------------- ---------------------- 3/7 [googleapis-common-protos]
   ---------------------- ----------------- 4/7 [beautifulsoup4]
   ---------------------------------------- 7/7 [google]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Initializing the Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os


pc = Pinecone(api_key='PINECONE_API_KEY')



pc.create_index(
    name="medical-chatbot",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-east-1'
    ) 
)




d:\healthcare chatbot\End-to-end-Medical-Chatbot-using-Llama2\mchatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\healthcare chatbot\End-to-end-Medical-Chatbot-using-Llama2\mchatbot\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at db_data_2025-04.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


{
    "name": "medical-chatbot",
    "metric": "cosine",
    "host": "medical-chatbot-zwqyx4x.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [18]:
# Embed each chunk and upsert the embeddings into your Pinecone index.

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name='medical-chatbot',
    embedding=embeddings,
)


KeyboardInterrupt: 

In [13]:
# Load Existing index
from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name='medical-chatbot',
    embedding=embeddings
)


In [26]:
docsearch

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("What is Acne?")


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()




In [ ]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Load API Key from .env
load_dotenv()
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

# Set Together.ai config (OpenAI-compatible)
os.environ["OPENAI_API_BASE"] = "https://api.together.xyz/v1"
os.environ["OPENAI_API_KEY"] = TOGETHER_API_KEY

# Set up Together-compatible LLM
llm = ChatOpenAI(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",  # You can try others too
    temperature=0.4,
    max_tokens=512
)

# Prompt template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentences maximum and keep the answer concise.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# ⚠️ Make sure 'retriever' is defined before this (e.g. from Pinecone or FAISS)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)




In [1]:
# Example call
response = rag_chain.invoke({"input": "i feel bloated , what should i do?"})
print(response["answer"])

NameError: name 'rag_chain' is not defined